In [ ]:
!pip install imap-tools

In [ ]:
!pip install streamlit

##Conneting to Email Inbox

In [ ]:
# Importing required libraries
import imap_tools
from imap_tools import MailBox, AND
import pandas as pd

# Email credentials and server details
EMAIL = 'vigneshwarsah2153@gmail.com'
PASSWORD = 'opysycbkcoccqwws'
IMAP_SERVER = 'imap.gmail.com'  # or your mail server

# Fetching emails
messages = []
try:
    with MailBox(IMAP_SERVER).login(EMAIL, PASSWORD, 'INBOX') as mailbox:
        for msg in mailbox.fetch(AND(seen=False), limit=10):  # test with 10 emails
            messages.append({
                'subject': msg.subject,
                'from': msg.from_,
                'body': msg.text,
                'date': msg.date_str
            })
except Exception as e:
    print(f"An error occurred while fetching emails: {e}")
    # Consider adding more specific error handling for authentication or connection issues

# Create DataFrame
df = pd.DataFrame(messages)

# Display the first few rows of the DataFrame
display(df.head())

,subject,from,body,date
0,Degree Or Experience,student@internshala.com,"Hi SCR VIGNESHWAR, Best posts for you:\r\n\r\n...","Sun, 17 Nov 2024 14:38:47 +0000"
1,6+ new internships for Data Science profile,student@internshala.com,"Scr Vigneshwar, earn high stipend in your pref...","Sun, 17 Nov 2024 14:50:22 +0000"
2,"✉️ Student, a special gift for your career!",trainings@internshala.com,Internshala Trainings \r\n\r\n <https://lnk.up...,"Mon, 18 Nov 2024 19:51:18 +0530"
3,DeepThought Edutech Ventures is hiring Large L...,student@internshala.com,"Scr Vigneshwar, earn high stipend in your pref...","Mon, 18 Nov 2024 14:49:44 +0000"
4,Learn How to Build Trusted Relationships with ...,apacemarketing@mail.salesforce.com,Learn How to Build Trusted Relationships with ...,"Mon, 18 Nov 2024 23:37:10 -0600"


##Storing in SQLite

In [ ]:
import sqlite3

conn = sqlite3.connect('emails.db')
cursor = conn.cursor()

# Creating table
cursor.execute('''
CREATE TABLE IF NOT EXISTS emails (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    sender TEXT,
    subject TEXT,
    body TEXT,
    date TEXT,
    category TEXT
)
''')
conn.commit()

# Inserting  fetched emails
for msg in messages:
    cursor.execute("INSERT INTO emails (sender, subject, body, date, category) VALUES (?, ?, ?, ?, ?)",
                   (msg['from'], msg['subject'], msg['body'], msg['date'], 'Uncategorized'))
conn.commit()



##NLP Based Classification

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Sample training data
X_train = [
    # Support
    "I can't log into my account",
    "Forgot password and need help resetting it",
    "System error when uploading documents",
    "Issue with invoice not generating properly",

    # Sales
    "We are interested in your product pricing",
    "Request for a product demo and pricing details",
    "Looking to buy bulk licenses for our team",
    "Can you share the quotation and features?",

    # HR
    "Please find my resume attached for your review",
    "I'm applying for the software developer role",
    "Job application for marketing position",
    "Resume and cover letter for your HR department",

    # Spam
    "Congratulations! You've won a free iPhone",
    "Click here to claim your gift card",
    "You have been selected for a special offer",
    "Claim your lottery prize now"
]

y_train = [
    "Support", "Support", "Support", "Support",
    "Sales", "Sales", "Sales", "Sales",
    "HR", "HR", "HR", "HR",
    "Spam", "Spam", "Spam", "Spam"
]


model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])
model.fit(X_train, y_train)

# Apply classification to emails in DB
cursor.execute("SELECT id, body FROM emails WHERE category = 'Uncategorized'")
for row in cursor.fetchall():
    email_id, body = row
    category = model.predict([body])[0]
    cursor.execute("UPDATE emails SET category = ? WHERE id = ?", (category, email_id))
conn.commit()


##Visualizing in Streamlit

In [ ]:
%%writefile dashboard.py
import streamlit as st
import sqlite3
import pandas as pd

conn = sqlite3.connect('emails.db')
df = pd.read_sql_query("SELECT * FROM emails", conn)

st.title("📬 Email Analytics Dashboard")

st.metric("📧 Total Emails", len(df))
st.bar_chart(df['category'].value_counts())

st.subheader("📈 Recent Emails")
st.dataframe(df[['date', 'sender', 'subject', 'category']].sort_values(by='date', ascending=False))


Overwriting dashboard.py


In [ ]:
!streamlit run dashboard.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.222.251.168:8501

  Stopping...
  Stopping...


In [ ]:
from google.colab import files
files.download('emails.db')
files.download('dashboard.py')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
readme_content = """

# Email Inbox Intelligence System with ML-based Categorization & Dashboard

# Downloading Necessary Package

This project implements a smart email inbox analyzer that:
- Fetches emails from an inbox
- Automatically classifies them using a Machine Learning (ML) model
- Displays categorized insights in a visual dashboard

## Features

-  IMAP-based email fetching
-  Machine Learning classification (e.g., Support, HR, Spam, etc.)
-  Visual dashboard to explore email categories and statistics
-  Handles email metadata, content, and optional attachments

## Tech Stack

- **Frontend**: Streamlit (Dashboard Interface)
- **Backend**: Python, IMAP, SQLite (or any DB)
- **ML/NLP**: Scikit-learn, Pandas, TfidfVectorizer, Naive Bayes
- **Visualization**: Matplotlib / Dashboard Rendering Libraries

"""

with open("README.md", "w") as f:
    f.write(readme_content)

print("README.md generated successfully.")

README.md generated successfully.


In [ ]:
from google.colab import files
files.download('README.md')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>